In [1]:
import numpy as np
import pandas as pd

In [2]:
df=pd.read_csv('./data/preprocessed_Bank_Customer_Churn_Prediction.csv')

In [3]:
df


,credit_score,country,gender,age,tenure,balance,products_number,credit_card,active_member,estimated_salary,churn
0,619,0,1,42,2,0.00,1,1,1,101348.88,1
1,608,2,1,41,1,83807.86,1,0,1,112542.58,0
2,502,0,1,42,8,159660.80,3,1,0,113931.57,1
3,699,0,1,39,1,0.00,2,0,0,93826.63,0
4,850,2,1,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,0,0,39,5,0.00,2,1,0,96270.64,0
9996,516,0,0,35,10,57369.61,1,1,1,101699.77,0
9997,709,0,1,36,7,0.00,1,0,1,42085.58,1
9998,772,1,0,42,3,75075.31,2,1,0,92888.52,1


### XGBOOST 사용

In [4]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report


# X, y 분리
y = df['churn']  # 목표 변수
X = df.drop(columns=['churn'])  # 특징 변수

# 학습 및 테스트 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# XGBoost 모델 생성 및 학습
model = xgb.XGBClassifier(
    objective='binary:logistic', 
    eval_metric='logloss', 
    use_label_encoder=False, 
    random_state=42
)
model.fit(X_train, y_train)

# 예측
y_pred = model.predict(X_test)

# 평가
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, digits=4)

print(f'Accuracy: {accuracy:.4f}')
print('Classification Report:\n', report)


Accuracy: 0.8530
Classification Report:
               precision    recall  f1-score   support

           0     0.8802    0.9439    0.9109      2389
           1     0.6941    0.4975    0.5796       611

    accuracy                         0.8530      3000
   macro avg     0.7871    0.7207    0.7453      3000
weighted avg     0.8423    0.8530    0.8434      3000



c:\Users\Playdata\AppData\Local\anaconda3\envs\psystudy_env\Lib\site-packages\xgboost\core.py:158: UserWarning: [20:25:21] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


In [5]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report

# 데이터 준비
y = df['churn']
X = df.drop(columns=['churn'])

# 학습 및 테스트 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# 하이퍼파라미터 설정
params = {
    'objective': 'binary:logistic',
    'eval_metric': 'logloss',
    'random_state': 42,
    'learning_rate': 0.1,
    'max_depth': 5,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'n_estimators': 100
}

# ✅ XGBoost의 cv 기능 사용
cv_results = xgb.cv(
    params=params,
    dtrain=xgb.DMatrix(X_train, label=y_train),
    num_boost_round=100,
    nfold=5,
    stratified=True,
    metrics='logloss',
    early_stopping_rounds=10
)

# 최적 n_estimators 설정
best_n_estimators = len(cv_results)
params['n_estimators'] = best_n_estimators

# 최적 모델 학습
model = xgb.XGBClassifier(**params)
model.fit(X_train, y_train)

# 예측
y_pred = model.predict(X_test)

# 평가
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, digits=4)

print(f'Best n_estimators: {best_n_estimators}')
print(f'Accuracy: {accuracy:.4f}')
print('Classification Report:\n', report)


c:\Users\Playdata\AppData\Local\anaconda3\envs\psystudy_env\Lib\site-packages\xgboost\core.py:158: UserWarning: [20:25:21] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)


Best n_estimators: 59
Accuracy: 0.8673
Classification Report:
               precision    recall  f1-score   support

           0     0.8775    0.9686    0.9208      2389
           1     0.7934    0.4714    0.5914       611

    accuracy                         0.8673      3000
   macro avg     0.8355    0.7200    0.7561      3000
weighted avg     0.8604    0.8673    0.8537      3000



### LGB사용

In [6]:
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report


# X, y 분리
y = df['churn']  # 목표 변수
X = df.drop(columns=['churn'])  # 특징 변수

# 학습 및 테스트 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# LightGBM 데이터셋 생성
train_data = lgb.Dataset(X_train, label=y_train)
valid_data = lgb.Dataset(X_test, label=y_test, reference=train_data)  # 검증 데이터 설정

# LightGBM 모델 학습
params = {
    'objective': 'binary',
    'metric': 'binary_logloss',
    'verbosity': -1
}

# callbacks을 사용하여 early stopping 적용
model = lgb.train(params, train_data, num_boost_round=100,
                  valid_sets=[valid_data], valid_names=['valid'],
                  callbacks=[lgb.early_stopping(10), lgb.log_evaluation(10)])

# 예측
y_pred = model.predict(X_test)
y_pred = [1 if x > 0.5 else 0 for x in y_pred]

# 평가
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, digits=4)


print(f'Accuracy: {accuracy:.4f}')
print('Classification Report:\n', report)


Training until validation scores don't improve for 10 rounds
[10]	valid's binary_logloss: 0.363612
[20]	valid's binary_logloss: 0.337697
[30]	valid's binary_logloss: 0.32777
[40]	valid's binary_logloss: 0.324757
[50]	valid's binary_logloss: 0.323227
[60]	valid's binary_logloss: 0.322389
[70]	valid's binary_logloss: 0.322251
Early stopping, best iteration is:
[67]	valid's binary_logloss: 0.321905
Accuracy: 0.8670
Classification Report:
               precision    recall  f1-score   support

           0     0.8806    0.9636    0.9202      2389
           1     0.7746    0.4894    0.5998       611

    accuracy                         0.8670      3000
   macro avg     0.8276    0.7265    0.7600      3000
weighted avg     0.8590    0.8670    0.8550      3000



### LGB gridsearch사용

In [7]:
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report


# X, y 분리
y = df['churn']  # 목표 변수
X = df.drop(columns=['churn'])  # 특징 변수

# 학습 및 테스트 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# LightGBM 모델 객체 생성
model = lgb.LGBMClassifier(objective='binary', metric='binary_logloss', verbosity=-1)

# 하이퍼파라미터 검색 공간 정의
param_grid = {
    'num_leaves': [20, 31, 50],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.05, 0.1],
    'min_child_samples': [10, 20, 50],
    'subsample': [0.7, 0.8, 0.9],
    'colsample_bytree': [0.7, 0.8, 0.9]
}

# Grid Search 수행
grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    scoring='accuracy',
    cv=3,  # 3-폴드 교차검증
    verbose=1,
    n_jobs=-1
)

grid_search.fit(X_train, y_train)

# 최적 하이퍼파라미터 출력
print("Best parameters found:", grid_search.best_params_)

# 최적 하이퍼파라미터 적용하여 모델 재학습
best_model = grid_search.best_estimator_

# 예측
y_pred = best_model.predict(X_test)

# 평가
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred,digits=4)

print(f'Accuracy: {accuracy:.4f}')
print('Classification Report:\n', report)


Fitting 3 folds for each of 729 candidates, totalling 2187 fits
Best parameters found: {'colsample_bytree': 0.7, 'learning_rate': 0.05, 'max_depth': 7, 'min_child_samples': 20, 'num_leaves': 20, 'subsample': 0.7}
Accuracy: 0.8693
Classification Report:
               precision    recall  f1-score   support

           0     0.8775    0.9715    0.9221      2389
           1     0.8085    0.4697    0.5942       611

    accuracy                         0.8693      3000
   macro avg     0.8430    0.7206    0.7582      3000
weighted avg     0.8634    0.8693    0.8553      3000



---

# optuna 사용


In [8]:
import pandas as pd
import lightgbm as lgb
import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report



# X, y 분리
y = df['churn']  # 목표 변수
X = df.drop(columns=['churn'])  # 특징 변수

# 학습 및 테스트 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Optuna를 이용한 하이퍼파라미터 튜닝
def objective(trial):
    params = {
        'objective': 'binary',
        'metric': 'binary_logloss',
        'verbosity': -1,
        'num_leaves': trial.suggest_int('num_leaves', 20, 150),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.5, 1.0)
    }
    
    train_data = lgb.Dataset(X_train, label=y_train)
    valid_data = lgb.Dataset(X_test, label=y_test, reference=train_data)
    
    model = lgb.train(params, train_data, num_boost_round=100,
                      valid_sets=[valid_data], 
                      callbacks=[lgb.early_stopping(10), lgb.log_evaluation(10)])
    
    y_pred = model.predict(X_test)
    y_pred = [1 if x > 0.5 else 0 for x in y_pred]
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

# Optuna 실행
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=30)

# 최적 하이퍼파라미터 출력
print("Best parameters found:", study.best_params)

# 최적 하이퍼파라미터로 모델 학습
best_params = study.best_params
best_params.update({'objective': 'binary', 'metric': 'binary_logloss', 'verbosity': -1})

train_data = lgb.Dataset(X_train, label=y_train)
valid_data = lgb.Dataset(X_test, label=y_test, reference=train_data)

model = lgb.train(best_params, train_data, num_boost_round=100,
                  valid_sets=[valid_data], 
                  callbacks=[lgb.early_stopping(10), lgb.log_evaluation(10)])

# 최적 모델 예측
y_pred = model.predict(X_test)
y_pred = [1 if x > 0.5 else 0 for x in y_pred]

# 평가
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy:.4f}')
print('Classification Report:\n', report)


[I 2025-02-08 20:28:34,158] A new study created in memory with name: no-name-aecefbd0-5165-4efd-a226-68d48e291fcd
C:\Users\Playdata\AppData\Local\Temp\ipykernel_11100\965261689.py:24: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
C:\Users\Playdata\AppData\Local\Temp\ipykernel_11100\965261689.py:26: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
C:\Users\Playdata\AppData\Local\Temp\ipykernel_11100\965261689.py:27: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/ta

Training until validation scores don't improve for 10 rounds
[10]	valid_0's binary_logloss: 0.369068
[20]	valid_0's binary_logloss: 0.341788
[30]	valid_0's binary_logloss: 0.332273


[I 2025-02-08 20:28:34,394] Trial 0 finished with value: 0.868 and parameters: {'num_leaves': 102, 'max_depth': 15, 'learning_rate': 0.09392912877877257, 'min_child_samples': 65, 'subsample': 0.9572992481157327, 'colsample_bytree': 0.9674451813971876}. Best is trial 0 with value: 0.868.


[40]	valid_0's binary_logloss: 0.332453
Early stopping, best iteration is:
[30]	valid_0's binary_logloss: 0.332273
Training until validation scores don't improve for 10 rounds
[10]	valid_0's binary_logloss: 0.451064
[20]	valid_0's binary_logloss: 0.416115
[30]	valid_0's binary_logloss: 0.39456


C:\Users\Playdata\AppData\Local\Temp\ipykernel_11100\965261689.py:24: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
C:\Users\Playdata\AppData\Local\Temp\ipykernel_11100\965261689.py:26: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
C:\Users\Playdata\AppData\Local\Temp\ipykernel_11100\965261689.py:27: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.5, 1.0)


...

[60]	valid_0's binary_logloss: 0.327228
[70]	valid_0's binary_logloss: 0.326143
[80]	valid_0's binary_logloss: 0.325143
[90]	valid_0's binary_logloss: 0.325211
Early stopping, best iteration is:
[87]	valid_0's binary_logloss: 0.324904
Training until validation scores don't improve for 10 rounds
[10]	valid_0's binary_logloss: 0.375007
[20]	valid_0's binary_logloss: 0.340177
[30]	valid_0's binary_logloss: 0.330293
[40]	valid_0's binary_logloss: 0.325615
[50]	valid_0's binary_logloss: 0.325206
[60]	valid_0's binary_logloss: 0.324563
[70]	valid_0's binary_logloss: 0.325169
Early stopping, best iteration is:
[65]	valid_0's binary_logloss: 0.324377


C:\Users\Playdata\AppData\Local\Temp\ipykernel_11100\965261689.py:24: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
C:\Users\Playdata\AppData\Local\Temp\ipykernel_11100\965261689.py:26: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
C:\Users\Playdata\AppData\Local\Temp\ipykernel_11100\965261689.py:27: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.5, 1.0)
[I 

Training until validation scores don't improve for 10 rounds
[10]	valid_0's binary_logloss: 0.38317
[20]	valid_0's binary_logloss: 0.345634
[30]	valid_0's binary_logloss: 0.333763
[40]	valid_0's binary_logloss: 0.327725
[50]	valid_0's binary_logloss: 0.325608
[60]	valid_0's binary_logloss: 0.323326
[70]	valid_0's binary_logloss: 0.322349
[80]	valid_0's binary_logloss: 0.322504
Early stopping, best iteration is:
[78]	valid_0's binary_logloss: 0.322014
Training until validation scores don't improve for 10 rounds
[10]	valid_0's binary_logloss: 0.366736
[20]	valid_0's binary_logloss: 0.334499
[30]	valid_0's binary_logloss: 0.327533
[40]	valid_0's binary_logloss: 0.32351
[50]	valid_0's binary_logloss: 0.324398
Early stopping, best iteration is:
[40]	valid_0's binary_logloss: 0.32351


[I 2025-02-08 20:28:44,266] Trial 28 finished with value: 0.8706666666666667 and parameters: {'num_leaves': 110, 'max_depth': 5, 'learning_rate': 0.16441973874173985, 'min_child_samples': 57, 'subsample': 0.5628140741281913, 'colsample_bytree': 0.5298717668378827}. Best is trial 26 with value: 0.874.
C:\Users\Playdata\AppData\Local\Temp\ipykernel_11100\965261689.py:24: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
C:\Users\Playdata\AppData\Local\Temp\ipykernel_11100\965261689.py:26: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
C:\Users\Playdata\AppData\Local\

Training until validation scores don't improve for 10 rounds
[10]	valid_0's binary_logloss: 0.392928
[20]	valid_0's binary_logloss: 0.354851
[30]	valid_0's binary_logloss: 0.339598
[40]	valid_0's binary_logloss: 0.330307
[50]	valid_0's binary_logloss: 0.326968
[60]	valid_0's binary_logloss: 0.324682
[70]	valid_0's binary_logloss: 0.323002
[80]	valid_0's binary_logloss: 0.321737
[90]	valid_0's binary_logloss: 0.321293
[100]	valid_0's binary_logloss: 0.320692
Did not meet early stopping. Best iteration is:
[99]	valid_0's binary_logloss: 0.320609
Best parameters found: {'num_leaves': 111, 'max_depth': 5, 'learning_rate': 0.14212578337462897, 'min_child_samples': 70, 'subsample': 0.572966772315434, 'colsample_bytree': 0.5359158611519707}
Training until validation scores don't improve for 10 rounds
[10]	valid_0's binary_logloss: 0.375007
[20]	valid_0's binary_logloss: 0.340177
[30]	valid_0's binary_logloss: 0.330293
[40]	valid_0's binary_logloss: 0.325615
[50]	valid_0's binary_logloss: 0.32

In [9]:
import pandas as pd
import lightgbm as lgb
import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, precision_recall_curve


# X, y 분리
y = df['churn']  # 목표 변수
X = df.drop(columns=['churn'])  # 특징 변수

# 학습 및 테스트 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Optuna를 이용한 하이퍼파라미터 튜닝
def objective(trial):
    params = {
        'objective': 'binary',
        'metric': 'binary_logloss',
        'verbosity': -1,
        'num_leaves': trial.suggest_int('num_leaves', 20, 150),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0)
    }
    
    train_data = lgb.Dataset(X_train, label=y_train)
    valid_data = lgb.Dataset(X_test, label=y_test, reference=train_data)
    
    model = lgb.train(params, train_data, num_boost_round=100,
                      valid_sets=[valid_data], 
                      callbacks=[lgb.early_stopping(10), lgb.log_evaluation(10)])
    
    y_pred_proba = model.predict(X_test)
    precision, recall, thresholds = precision_recall_curve(y_test, y_pred_proba)
    best_threshold = thresholds[(precision + recall).argmax()]  # 최적의 threshold 선택
    
    y_pred = [1 if x > best_threshold else 0 for x in y_pred_proba]
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

# Optuna 실행
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=30)

# 최적 하이퍼파라미터 출력
print("Best parameters found:", study.best_params)

# 최적 하이퍼파라미터로 모델 학습
best_params = study.best_params
best_params.update({'objective': 'binary', 'metric': 'binary_logloss', 'verbosity': -1})

train_data = lgb.Dataset(X_train, label=y_train)
valid_data = lgb.Dataset(X_test, label=y_test, reference=train_data)

model = lgb.train(best_params, train_data, num_boost_round=100,
                  valid_sets=[valid_data], 
                  callbacks=[lgb.early_stopping(10), lgb.log_evaluation(10)])

# 최적 모델 예측
y_pred_proba = model.predict(X_test)
precision, recall, thresholds = precision_recall_curve(y_test, y_pred_proba)
best_threshold = thresholds[(precision + recall).argmax()]  # 최적 threshold 선택

print(f'Best Threshold: {best_threshold:.4f}')

y_pred = [1 if x > best_threshold else 0 for x in y_pred_proba]

# 평가
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy:.4f}')
print('Classification Report:\n', report)


[I 2025-02-08 20:28:44,613] A new study created in memory with name: no-name-93807782-b866-4f11-af54-11c58d06e3e5
[I 2025-02-08 20:28:44,690] Trial 0 finished with value: 0.803 and parameters: {'num_leaves': 50, 'max_depth': 3, 'learning_rate': 0.12467137122952576, 'min_child_samples': 14, 'subsample': 0.6925485564728842, 'colsample_bytree': 0.8635862555703362}. Best is trial 0 with value: 0.803.


Training until validation scores don't improve for 10 rounds
[10]	valid_0's binary_logloss: 0.377416
[20]	valid_0's binary_logloss: 0.348973
[30]	valid_0's binary_logloss: 0.335884
[40]	valid_0's binary_logloss: 0.329403
[50]	valid_0's binary_logloss: 0.325272
[60]	valid_0's binary_logloss: 0.323029
[70]	valid_0's binary_logloss: 0.321191
[80]	valid_0's binary_logloss: 0.320538
[90]	valid_0's binary_logloss: 0.320628
Early stopping, best iteration is:
[80]	valid_0's binary_logloss: 0.320538
Training until validation scores don't improve for 10 rounds
[10]	valid_0's binary_logloss: 0.38868
[20]	valid_0's binary_logloss: 0.3493
[30]	valid_0's binary_logloss: 0.33609
[40]	valid_0's binary_logloss: 0.329117
[50]	valid_0's binary_logloss: 0.326996
[60]	valid_0's binary_logloss: 0.324579
[70]	valid_0's binary_logloss: 0.323445
[80]	valid_0's binary_logloss: 0.323076
[90]	valid_0's binary_logloss: 0.322573


[I 2025-02-08 20:28:44,823] Trial 1 finished with value: 0.792 and parameters: {'num_leaves': 34, 'max_depth': 5, 'learning_rate': 0.11272648215039936, 'min_child_samples': 81, 'subsample': 0.7991249686556032, 'colsample_bytree': 0.5163253187975734}. Best is trial 0 with value: 0.803.


[100]	valid_0's binary_logloss: 0.323415
Did not meet early stopping. Best iteration is:
[91]	valid_0's binary_logloss: 0.322444
Training until validation scores don't improve for 10 rounds
[10]	valid_0's binary_logloss: 0.424888
[20]	valid_0's binary_logloss: 0.385597
[30]	valid_0's binary_logloss: 0.362932
[40]	valid_0's binary_logloss: 0.349903
[50]	valid_0's binary_logloss: 0.34233
[60]	valid_0's binary_logloss: 0.337259
[70]	valid_0's binary_logloss: 0.334117
[80]	valid_0's binary_logloss: 0.331963


[I 2025-02-08 20:28:45,374] Trial 2 finished with value: 0.824 and parameters: {'num_leaves': 74, 'max_depth': 10, 'learning_rate': 0.03589874859797542, 'min_child_samples': 32, 'subsample': 0.8029875250555134, 'colsample_bytree': 0.8740876259703843}. Best is trial 2 with value: 0.824.


...

Training until validation scores don't improve for 10 rounds
[10]	valid_0's binary_logloss: 0.473482
[20]	valid_0's binary_logloss: 0.449423
[30]	valid_0's binary_logloss: 0.428336
[40]	valid_0's binary_logloss: 0.412334
[50]	valid_0's binary_logloss: 0.400005
[60]	valid_0's binary_logloss: 0.388571
[70]	valid_0's binary_logloss: 0.378814


[I 2025-02-08 20:28:53,497] Trial 28 finished with value: 0.798 and parameters: {'num_leaves': 97, 'max_depth': 7, 'learning_rate': 0.015036772552550206, 'min_child_samples': 76, 'subsample': 0.7154311547847175, 'colsample_bytree': 0.7330263520316256}. Best is trial 21 with value: 0.8753333333333333.


[80]	valid_0's binary_logloss: 0.369447
[90]	valid_0's binary_logloss: 0.362788
[100]	valid_0's binary_logloss: 0.357349
Did not meet early stopping. Best iteration is:
[100]	valid_0's binary_logloss: 0.357349
Training until validation scores don't improve for 10 rounds
[10]	valid_0's binary_logloss: 0.435563
[20]	valid_0's binary_logloss: 0.395001
[30]	valid_0's binary_logloss: 0.369955
[40]	valid_0's binary_logloss: 0.354206


[I 2025-02-08 20:28:53,795] Trial 29 finished with value: 0.8246666666666667 and parameters: {'num_leaves': 43, 'max_depth': 11, 'learning_rate': 0.04037293805980678, 'min_child_samples': 90, 'subsample': 0.5524603776891457, 'colsample_bytree': 0.6640869501191893}. Best is trial 21 with value: 0.8753333333333333.


[50]	valid_0's binary_logloss: 0.344381
[60]	valid_0's binary_logloss: 0.337458
[70]	valid_0's binary_logloss: 0.332858
[80]	valid_0's binary_logloss: 0.328773
[90]	valid_0's binary_logloss: 0.326881
[100]	valid_0's binary_logloss: 0.325621
Did not meet early stopping. Best iteration is:
[100]	valid_0's binary_logloss: 0.325621
Best parameters found: {'num_leaves': 75, 'max_depth': 9, 'learning_rate': 0.04728923452292558, 'min_child_samples': 75, 'subsample': 0.6279748133999552, 'colsample_bytree': 0.5748630153653248}
Training until validation scores don't improve for 10 rounds
[10]	valid_0's binary_logloss: 0.431201
[20]	valid_0's binary_logloss: 0.389801
[30]	valid_0's binary_logloss: 0.368494
[40]	valid_0's binary_logloss: 0.351174
[50]	valid_0's binary_logloss: 0.342535
[60]	valid_0's binary_logloss: 0.336761
[70]	valid_0's binary_logloss: 0.33219
[80]	valid_0's binary_logloss: 0.328773
[90]	valid_0's binary_logloss: 0.326949
[100]	valid_0's binary_logloss: 0.326341
Did not meet ea

Precision과 Recall 균형 조정

Precision 증가(0.80 → 0.84)
Recall 감소(0.48 → 0.44)
이탈 고객을 보다 정확하게 예측하지만, 일부를 놓칠 가능성이 있음
비즈니스 목표에 따라 Threshold를 조정할 필요 있음!
→ 이탈 고객 Recall을 높이고 싶다면 더 낮은 Threshold를 시도하는 것도 가능